In [1]:
import lazypredict

# Read dataframe

In [2]:
import sys
import numpy
numpy.set_printoptions(threshold=sys.maxsize)

#Read data file

import pandas as pd

filepath = "dataset/balanced_dataset2.csv"
df = pd.read_csv(filepath)
df

,page_id,page_title,view_count,protection_status,page_length,edit_count,page_watchers,page_watchers_recent_edits,page_views_past_30days,page_id_scrapped,redirects,recent_number_of_edits,number_distinct_authors,number_categories
0,25869317,Mercosur,1770104,unprotected,90427,1785.0,194,20,13584,25869317,18,1.0,1.0,42.0
1,67728,Petrochemical,1486571,unprotected,23015,554.0,135,14,14823,67728,5,1.0,1.0,7.0
2,1914,Antimicrobial resistance,1461174,unprotected,132431,3367.0,264,26,8607,1914,29,7.0,6.0,10.0
3,855728,Red McCombs,485956,unprotected,14842,375.0,32,2,4808,855728,2,0.0,0.0,12.0
4,5508002,New World crops,442438,unprotected,12901,343.0,46,4,9433,5508002,5,3.0,2.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
828,11578887,General Hospital cast members,1564023,autoconfirmed,68,1.0,112,65,634,59351114,0,0.0,0.0,NaN
829,1160865,Samy Vellu,208303,autoconfirmed,20539,986.0,Fewer than 30 watchers,NaN,3743,1160865,6,0.0,0.0,16.0
830,1164274,Claremont Graduate University,401824,autoconfirmed,18822,592.0,45,4,3327,1164274,11,0.0,0.0,18.0
831,11650319,Xiaxue,237120,autoconfirmed,14134,1521.0,46,3,1677,11650319,2,0.0,0.0,9.0


# Prepare the data (pre-processing)

In [3]:
# Use numpy to convert to arrays
import numpy as np

features = df

# Labels are the values we want to predict
labels = np.array(df['protection_status'])

# Remove the labels from the features
# axis 1 refers to the columns
features = features.drop('protection_status', axis = 1)
features = features.drop('page_title', axis = 1)
features = features.drop('page_id', axis=1)

#Mapping for encoding
# Page Watchers column: -1 => "Fewer than 30 watchers"
# page_watchers_recent_edits column: "-2 => "There may or may not be a watching user visiting recent edits"
# All Columns => -5: NaN    

    
#Encode string to float
features.loc[features['page_watchers'] == "Fewer than 30 watchers", 'page_watchers'] = -1
features.loc[features['page_watchers_recent_edits'] == "There may or may not be a watching user visiting recent edits", 'page_watchers_recent_edits'] = -2
features = features.fillna(-5)

#Convert String to Floats
features['page_length'] = features['page_length'].astype(float)
features['edit_count'] = features['edit_count'].astype(float)
features['page_watchers'] = features['page_watchers'].astype(float)
features['page_watchers_recent_edits'] = features['page_watchers_recent_edits'].astype(float)

# Saving feature names for later use
feature_list = list(features.columns)

# Convert to numpy array
features = np.array(features)

## Encode categorical values

In [4]:
#Label encoding for protection_status column

# 0 => unprotected
# 1 => autoconfirmed
# 2 => extendedconfirmed
# 3 => sysop
labels_encoded = []
for item in labels:
    if(item =="unprotected"):
        labels_encoded.append(0)
    elif(item == "autoconfirmed"):
        labels_encoded.append(1)
    elif(item == "extendedconfirmed"):
        labels_encoded.append(2)
    elif(item == "sysop"):
        labels_encoded.append(3)    
        

# Model training and results

In [5]:
# Using Skicit-learn to split data into training and testing sets
from sklearn.model_selection import train_test_split
# Split the data into training and testing sets
train_features, test_features, train_labels, test_labels = train_test_split(features, labels_encoded, test_size = 0.20, random_state = 25)

In [9]:
X_train = train_features
y_train = train_labels
X_test = test_features
y_test = test_labels
from lazypredict.Supervised import LazyClassifier
clf = LazyClassifier(verbose=0,ignore_warnings=True, custom_metric=None)
models,predictions = clf.fit(X_train, X_test, y_train, y_test)

print(models)

100%|██████████████████████████████████████████████████████████████████████████████████| 29/29 [00:02<00:00, 12.79it/s]

                               Accuracy  Balanced Accuracy ROC AUC  F1 Score  \
Model                                                                          
XGBClassifier                      0.79               0.79    None      0.79   
LGBMClassifier                     0.79               0.79    None      0.79   
BaggingClassifier                  0.75               0.75    None      0.74   
RandomForestClassifier             0.74               0.74    None      0.73   
ExtraTreesClassifier               0.72               0.72    None      0.72   
DecisionTreeClassifier             0.71               0.71    None      0.71   
AdaBoostClassifier                 0.63               0.63    None      0.63   
ExtraTreeClassifier                0.60               0.60    None      0.59   
LogisticRegression                 0.57               0.58    None      0.56   
SVC                                0.57               0.57    None      0.56   
LabelSpreading                     0.55 